# Top-k (no econ filtering)

In [6]:
DATA_PATH = 'data/'

PAIRWISE_MAPPING_FILE_NAME = 'tables_OZ_geo_5500/processed/regex-pairwise-groups/regex-pairwise-groups_num-queries=20_patterns-dict-hash=a6223255f273e52a893ba7235e3c19b3/mapping.parquet'
SOURCE_FILEPATH = 'tables_OZ_geo_5500/processed/OZ_geo_5500.csv'

QUERY_SELLER = 'ИНТЕРТРЕЙД'

In [ ]:
# Create all < query, candidate > pairs w / labels
# TODO: move explicit regex pairwise dataset creation to make_OZ_geo_5500_pairwise-regex.ipynb

from pathlib import Path
import pandas as pd
from tqdm import tqdm

source_df = pd.read_csv(DATA_PATH + SOURCE_FILEPATH)
mapping_df = pd.read_parquet(DATA_PATH + PAIRWISE_MAPPING_FILE_NAME)
pairwise_queries_file_name = Path(PAIRWISE_MAPPING_FILE_NAME).parent / 'pairwise-queries.parquet'

if (DATA_PATH / pairwise_queries_file_name).exists():
    print(f"Pairwise queries file exists. Loading it from \n{pairwise_queries_file_name}")
    pairwise_df = pd.read_parquet(DATA_PATH / pairwise_queries_file_name)
else:
    print(f"Pairwise queries file does not exist. Creating it at \n{pairwise_queries_file_name}")
    query_df = source_df[source_df['seller'] == QUERY_SELLER]
    query_df = query_df[query_df['sku'].isin(mapping_df['sku_query'])]  # drop all queries without positive pairs
    non_query_df = source_df[source_df['seller'] != QUERY_SELLER]

    pairwise_rows = []

    def make_pair(query_row, candidate_row, label, pair_type):
        pair = {}
        for col in query_row.index:
            pair[f"{col}_first"] = query_row[col]
        for col in candidate_row.index:
            pair[f"{col}_second"] = candidate_row[col]
        pair['label'] = label
        pair['pair_type'] = pair_type
        return pair

    def add_pairs(query_row, sku_list, label, pair_type):
        if not isinstance(sku_list, (list, np.ndarray)):
            sku_list = [sku_list]
        for sku in sku_list:
            candidate_row = non_query_df[non_query_df['sku'] == sku]
            if candidate_row.empty:
                continue
            candidate_row = candidate_row.iloc[0]
            pairwise_rows.append(make_pair(query_row, candidate_row, label, pair_type))

    for idx, row in tqdm(mapping_df.iterrows(), desc='sku_query', total=len(mapping_df), leave=True):
        query_sku = row['sku_query']
        query_row = query_df[query_df['sku'] == query_sku]
        if query_row.empty:
            continue
        query_row = query_row.iloc[0]

        add_pairs(query_row, row['sku_pos'], 1, 'sku_pos')
        add_pairs(query_row, row.get('sku_hard_neg', []), 0, 'sku_hard_neg')
        add_pairs(query_row, row.get('sku_soft_neg', []), 0, 'sku_soft_neg')

    pairwise_df = pd.DataFrame(pairwise_rows)
    pairwise_df.to_parquet(DATA_PATH / pairwise_queries_file_name)
    print(f"Total number of <query, candidate> pairs: {len(pairwise_df)}")

Pairwise queries file exists. Loading it from 
tables_OZ_geo_5500/processed/regex-pairwise-groups/regex-pairwise-groups_num-queries=20_patterns-dict-hash=a6223255f273e52a893ba7235e3c19b3/pairwise-queries.parquet


In [11]:
def construct_wide_table(
    df,
    label_col: str,
    top_k: int = None,
    positive_only: bool = True,
    include_urls: bool = False,
    layout: str = 'blocked',
):
    """
    Builds a wide-format table of top‐k matches per Query_SKU.
    """
    rows = []
    # Determine how many slots: either fixed top_k or max found per group
    if top_k is None:
        max_counts = (
            df[df[label_col] == 1]
            .groupby('sku_first')
            .size()
            .max()
        )
        n_slots = int(max_counts)
    else:
        n_slots = int(top_k)

    for query_sku, group in df.groupby('sku_first'):
        candidates = group[group[label_col] == 1] if positive_only else group

        # always reset index so positional iloc works
        candidates = candidates.reset_index(drop=True)

        # then sort by probability if available
        if 'proba' in candidates.columns:
            candidates = (
                candidates.sort_values('proba', ascending=False)
                          .reset_index(drop=True)
            )

        row = {'Query_SKU': int(query_sku)}

        for i in range(n_slots):
            if i < len(candidates):
                row[f'Top-{i+1}_SKU'] = int(candidates.iloc[i]['sku_second'])
                if include_urls and 'url_second' in candidates.columns:
                    row[f'Top-{i+1}_URL'] = candidates.iloc[i]['url_second']
                if 'proba' in candidates.columns:
                    row[f'Top-{i+1}_Proba'] = float(candidates.iloc[i]['proba'])
            else:
                row[f'Top-{i+1}_SKU'] = -1
                if include_urls:
                    row[f'Top-{i+1}_URL'] = ''
                if 'proba' in candidates.columns:
                    row[f'Top-{i+1}_Proba'] = 0.0

        rows.append(row)

    matches_wide_df = pd.DataFrame(rows)

    if layout == 'blocked':
        sku_cols   = [c for c in matches_wide_df if c.startswith('Top-') and c.endswith('_SKU')]
        url_cols   = [c for c in matches_wide_df if c.startswith('Top-') and c.endswith('_URL')]
        proba_cols = [c for c in matches_wide_df if c.startswith('Top-') and c.endswith('_Proba')]
        matches_wide_df = matches_wide_df[['Query_SKU'] + sku_cols + url_cols + proba_cols]

    return matches_wide_df


In [12]:
# get matches for unfiltered data

all_matches_wide_df = construct_wide_table(
    df=pairwise_df,
    label_col = 'label',
    top_k = None,
    positive_only=True,
    include_urls=False,
    layout='blocked',
)
all_matches_wide_df.dropna(inplace=True)
pd.set_option('display.max_columns', None)  # 'max_cols' is also valid
all_matches_wide_df

,Query_SKU,Top-1_SKU,Top-2_SKU,Top-3_SKU,Top-4_SKU,Top-5_SKU,Top-6_SKU,Top-7_SKU,Top-8_SKU,Top-9_SKU,Top-10_SKU,Top-11_SKU,Top-12_SKU,Top-13_SKU,Top-14_SKU,Top-15_SKU,Top-16_SKU,Top-17_SKU,Top-18_SKU,Top-19_SKU,Top-20_SKU,Top-21_SKU,Top-22_SKU,Top-23_SKU,Top-24_SKU,Top-25_SKU,Top-26_SKU,Top-27_SKU,Top-28_SKU,Top-29_SKU,Top-30_SKU,Top-31_SKU,Top-32_SKU,Top-33_SKU,Top-34_SKU,Top-35_SKU,Top-36_SKU,Top-37_SKU,Top-38_SKU,Top-39_SKU,Top-40_SKU,Top-41_SKU,Top-42_SKU,Top-43_SKU,Top-44_SKU,Top-45_SKU,Top-46_SKU,Top-47_SKU,Top-48_SKU,Top-49_SKU,Top-50_SKU,Top-51_SKU,Top-52_SKU,Top-53_SKU,Top-54_SKU,Top-55_SKU,Top-56_SKU,Top-57_SKU,Top-58_SKU,Top-59_SKU,Top-60_SKU,Top-61_SKU,Top-62_SKU,Top-63_SKU,Top-64_SKU,Top-65_SKU,Top-66_SKU,Top-67_SKU,Top-68_SKU,Top-69_SKU,Top-70_SKU,Top-71_SKU,Top-72_SKU,Top-73_SKU,Top-74_SKU,Top-75_SKU,Top-76_SKU,Top-77_SKU,Top-78_SKU,Top-79_SKU,Top-80_SKU,Top-81_SKU,Top-82_SKU,Top-83_SKU,Top-84_SKU,Top-85_SKU,Top-86_SKU,Top-87_SKU,Top-88_SKU,Top-89_SKU,Top-90_SKU,Top-91_SKU,Top-92_SKU,Top-93_SKU,Top-94_SKU,Top-95_SKU,Top-96_SKU,Top-97_SKU,Top-98_SKU,Top-99_SKU,Top-100_SKU,Top-101_SKU,Top-102_SKU,Top-103_SKU,Top-104_SKU,Top-105_SKU,Top-106_SKU,Top-107_SKU,Top-108_SKU,Top-109_SKU,Top-110_SKU,Top-111_SKU,Top-112_SKU,Top-113_SKU,Top-114_SKU,Top-115_SKU,Top-116_SKU,Top-117_SKU,Top-118_SKU,Top-119_SKU,Top-120_SKU,Top-121_SKU,Top-122_SKU,Top-123_SKU,Top-124_SKU,Top-125_SKU,Top-126_SKU,Top-127_SKU,Top-128_SKU,Top-129_SKU,Top-130_SKU,Top-131_SKU,Top-132_SKU,Top-133_SKU,Top-134_SKU,Top-135_SKU,Top-136_SKU,Top-137_SKU,Top-138_SKU,Top-139_SKU,Top-140_SKU,Top-141_SKU,Top-142_SKU,Top-143_SKU,Top-144_SKU,Top-145_SKU,Top-146_SKU,Top-147_SKU,Top-148_SKU,Top-149_SKU,Top-150_SKU,Top-151_SKU,Top-152_SKU,Top-153_SKU,Top-154_SKU,Top-155_SKU,Top-156_SKU,Top-157_SKU,Top-158_SKU,Top-159_SKU,Top-160_SKU,Top-161_SKU,Top-162_SKU,Top-163_SKU,Top-164_SKU,Top-165_SKU,Top-166_SKU,Top-167_SKU,Top-168_SKU,Top-169_SKU,Top-170_SKU,Top-171_SKU,Top-172_SKU,Top-173_SKU,Top-174_SKU,Top-175_SKU,Top-176_SKU,Top-177_SKU,Top-178_SKU,Top-179_SKU,Top-180_SKU,Top-181_SKU,Top-182_SKU,Top-183_SKU,Top-184_SKU,Top-185_SKU,Top-186_SKU,Top-187_SKU,Top-188_SKU,Top-189_SKU,Top-190_SKU,Top-191_SKU,Top-192_SKU,Top-193_SKU,Top-194_SKU,Top-195_SKU,Top-196_SKU,Top-197_SKU,Top-198_SKU,Top-199_SKU,Top-200_SKU,Top-201_SKU,Top-202_SKU,Top-203_SKU,Top-204_SKU,Top-205_SKU,Top-206_SKU,Top-207_SKU,Top-208_SKU,Top-209_SKU,Top-210_SKU,Top-211_SKU,Top-212_SKU,Top-213_SKU,Top-214_SKU,Top-215_SKU,Top-216_SKU,Top-217_SKU,Top-218_SKU,Top-219_SKU,Top-220_SKU,Top-221_SKU,Top-222_SKU,Top-223_SKU,Top-224_SKU,Top-225_SKU,Top-226_SKU,Top-227_SKU,Top-228_SKU,Top-229_SKU,Top-230_SKU,Top-231_SKU,Top-232_SKU,Top-233_SKU,Top-234_SKU,Top-235_SKU,Top-236_SKU,Top-237_SKU,Top-238_SKU,Top-239_SKU,Top-240_SKU,Top-241_SKU,Top-242_SKU,Top-243_SKU,Top-244_SKU,Top-245_SKU,Top-246_SKU,Top-247_SKU,Top-248_SKU,Top-249_SKU,Top-250_SKU,Top-251_SKU,Top-252_SKU,Top-253_SKU,Top-254_SKU,Top-255_SKU,Top-256_SKU,Top-257_SKU,Top-258_SKU,Top-259_SKU,Top-260_SKU,Top-261_SKU,Top-262_SKU,Top-263_SKU,Top-264_SKU,Top-265_SKU,Top-266_SKU,Top-267_SKU,Top-268_SKU,Top-269_SKU,Top-270_SKU,Top-271_SKU,Top-272_SKU
0,491268805,1176719536,804154003,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1

In [13]:
# save matches and pairs for unfiltered data

from pathlib import Path

SAVE_DIR = Path('data/tables_OZ_geo_5500/test_results')
INCLUDE_PAIR_COLS = [
    'sku_first', 'sku_second',
    'name_first', 'name_second',
    'final_price_first', 'final_price_second',
    'balance_fbo_first', 'balance_fbo_second',
    'balance_fbs_first', 'balance_fbs_second',
    'rating_first', 'rating_second',
    'description_first', 'description_second'
]

all_matches_wide_df.to_csv(SAVE_DIR / 'regex-evaluation_all-matches.csv', index=False)
pairwise_df[INCLUDE_PAIR_COLS].to_csv(SAVE_DIR / 'regex-evaluation_all-matches-paired.csv', index=False)

# Top-k w/ econ filtering

In [14]:
# Filter by economic features

PRICE_MARGIN = 0.3
MIN_SALES = 0
MIN_FBO = 0
MIN_RATING = 4.3

# --- original filtering mask ---
mask = (
    (pairwise_df['sales_second'] > MIN_SALES)
    & ((pairwise_df['balance_fbo_second'] > MIN_FBO) | (pairwise_df['balance_fbs_second'] > MIN_FBO))
    & (pairwise_df['rating_second'] > MIN_RATING)
)
filtered = pairwise_df[mask]

# --- now per‐sku_first price‐ratio filtering ---
def within_margin(group):
    ratio = group['final_price_second'] / group['final_price_first']
    return group[ratio.between(1 - PRICE_MARGIN, 1 + PRICE_MARGIN)]

filtered_pairwise_df = (
    filtered
    .groupby('sku_first', group_keys=False)
    .apply(within_margin)
)

print('Original size:', len(pairwise_df))
print('Filtered size:', len(filtered_pairwise_df))


Original size: 110780
Filtered size: 1149


/tmp/ipykernel_12474/1017522135.py:24: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(within_margin)


In [15]:
# display & save filtered data

filtered_matches_wide_df = construct_wide_table(
    df=filtered_pairwise_df,
    label_col = 'label',
    top_k = None,
    positive_only=True,
    include_urls=False,
    layout='blocked',
)

# Drop rows with all -1
cols_to_check = list(set(filtered_matches_wide_df.columns) - {'Query_SKU'})
filtered_matches_wide_df = filtered_matches_wide_df[
    ~(filtered_matches_wide_df[cols_to_check] == -1).all(axis=1)
]

pd.set_option('display.max_columns', None)  # 'max_cols' is also valid
filtered_matches_wide_df

,Query_SKU,Top-1_SKU,Top-2_SKU,Top-3_SKU,Top-4_SKU,Top-5_SKU,Top-6_SKU,Top-7_SKU,Top-8_SKU,Top-9_SKU,Top-10_SKU,Top-11_SKU
0,491268805,804154003,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
2,491270369,1857130100,1737112763,1737112217,219077186,178726257,-1,-1,-1,-1,-1,-1
3,491271284,1758403711,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
4,491271320,1666035693,861593242,268682160,-1,-1,-1,-1,-1,-1,-1,-1
5,491271768,1711547819,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
6,491273438,1317729731,1190097076,-1,-1,-1,-1,-1,-1,-1,-1,-1
7,491273791,1758403630,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
8,491279127,1713026634,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1
11,508611672,847687475,847684763,-1,-1,-1,-1,-1,-1,-1,-1,-1
13,824158517,1713036930,861605997,856647774,851295373,844750071,671211264,601320601,490461399,-1,-1,-1


In [16]:
# save matches and pairs for filtered data

from pathlib import Path

SAVE_DIR = Path('data/tables_OZ_geo_5500/test_results')
INCLUDE_PAIR_COLS = [
    'sku_first', 'sku_second',
    'name_first', 'name_second',
    'final_price_first', 'final_price_second',
    'balance_fbo_first', 'balance_fbo_second',
    'balance_fbs_first', 'balance_fbs_second',
    'rating_first', 'rating_second',
    'description_first', 'description_second'
]

filtered_matches_wide_df.to_csv(SAVE_DIR / 'regex-evaluation_filtered-matches.csv', index=False)
filtered_pairwise_df[INCLUDE_PAIR_COLS].to_csv(SAVE_DIR / 'regex-evaluation_filtered-matches-paired.csv', index=False)